In [3]:
pip install easyocr


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 912.2/912.2 kB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.8/286.8 kB 21.1 MB/s eta 0:00:00


In [4]:
import os
import easyocr

# Initialize the EasyOCR Reader (supports multiple languages, e.g., ['en'] for English)
reader = easyocr.Reader(['en'])

# Define the path to your dataset
dataset_path = '/content/drive/MyDrive/dataset'

# Function to extract text from front and back images of a product
def extract_product_details(product_folder):
    front_image_path = os.path.join(product_folder, 'front.jpg')  # Updated to .jpg format
    back_image_path = os.path.join(product_folder, 'back.jpg')    # Updated to .jpg format

    # Check if the images exist
    if not os.path.exists(front_image_path) or not os.path.exists(back_image_path):
        print(f"Images missing in {product_folder}")
        return None

    # Reading text from front and back images
    front_text = reader.readtext(front_image_path, detail=0)  # detail=0 returns just the text
    back_text = reader.readtext(back_image_path, detail=0)

    # Combining text from both images
    combined_text = {
        "product_name": os.path.basename(product_folder),
        "front_image_text": front_text,
        "back_image_text": back_text
    }

    return combined_text

# Iterate through the dataset and process each product folder
def process_dataset(dataset_path):
    product_details = []

    # List all product folders
    for product_name in os.listdir(dataset_path):
        product_folder = os.path.join(dataset_path, product_name)
        if os.path.isdir(product_folder):
            details = extract_product_details(product_folder)
            if details:
                product_details.append(details)

    return product_details

# Run the function on your dataset
product_data = process_dataset(dataset_path)

# Example of how you can display the extracted data
for product in product_data:
    print(f"Product: {product['product_name']}")
    print(f"Front Image Text: {product['front_image_text']}")
    print(f"Back Image Text: {product['back_image_text']}")
    print("-" * 50)


Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% CompleteImages missing in /content/drive/MyDrive/dataset/complain
Product: bread
Front Image Text: ['Huvzaa|', 'ZERO', 'TRANS', 'FAT', '4GLIS,', 'FIBER HELPS', 'DIGESTION', 'OVE)', 'BROWN', 'HIGH', 'FIBRE BREAD', 'FRESHLY BAKED']
Back Image Text: ['Huvzaa|', 'COMMODITY NAME: BROWN BREAD', 'NUTRITONAL INFORMATION', '7.1.1 [Bread and Rolls]', 'PER 100g PRODUCT (APPROX ,', 'INGREDIENTS: Weat Roulma da]', 'Waler   Yeast  Whzat', 'Eneagy', 'Zjoacal', 'Ala(5%0} Sugar', 'Edibe? Conmcn Salt, Ecble Vegetatle', 'carbohydaATES', '53.t1g', 'Ois (Palm}', 'Guren', 'Bran, Roasted Cerea', 'Qf WHICH SUGARS', '2.059', 'Flour;   Soya  Flur;   Class', 'Preservabive/2821,', 'Acidily', 'Reg lator (260]  Emulstiers [471, 472e, 481f], Improve 5', 'SATURATED FAT', '0,26g', '[170 @,510,923,1100] And  Antiowdart (3001.', 'TRANS FaT', 'PROTEIN', '7,930', '(NUMBERS IN BRACKETS ARE AS PER', 'dietary fiber', 'INTERNATIONAL NUMBERING SYSTEM)', 'ALL

In [5]:
import os
import easyocr
import re
from datetime import datetime
from dateutil.relativedelta import relativedelta  # To add months easily

# Initialize the EasyOCR Reader
reader = easyocr.Reader(['en'])

# Function to extract MRP, expiry date, and best-before details
def extract_mrp_and_expiry(text):
    mrp = None
    expiry_date = None
    packing_date = None
    best_before_months = None

    # Regex patterns
    mrp_pattern = r'MRP\s*:?[\s₹]*\d+\.?\d*'  # MRP pattern
    expiry_pattern = r'(expiry|exp|use by|best before)\s*:?[\w\s]*\d{1,2}[\/\-]\d{1,2}[\/\-]\d{2,4}'  # Direct expiry date
    best_before_pattern = r'best before\s*(\d+)\s*months'  # Best before N months
    packing_date_pattern = r'packing date\s*:?[\w\s]*\d{1,2}[\/\-]\d{1,2}[\/\-]\d{2,4}'  # Packing date

    # Join the text into a single string for regex matching
    full_text = " ".join(text).lower()

    # Search for MRP
    mrp_match = re.search(mrp_pattern, full_text)
    if mrp_match:
        mrp = mrp_match.group()

    # Search for direct expiry date
    expiry_match = re.search(expiry_pattern, full_text)
    if expiry_match:
        expiry_date = expiry_match.group()

    # Search for 'Best before N months'
    best_before_match = re.search(best_before_pattern, full_text)
    if best_before_match:
        best_before_months = int(best_before_match.group(1))  # Extract number of months

    # Search for packing date
    packing_date_match = re.search(packing_date_pattern, full_text)
    if packing_date_match:
        packing_date_str = packing_date_match.group()
        # Try to parse the packing date (e.g., 'Packing Date: 12/12/2023')
        packing_date = parse_date_from_string(packing_date_str)

    return mrp, expiry_date, best_before_months, packing_date

# Helper function to parse a date from a string
def parse_date_from_string(date_str):
    try:
        # Try to parse in different formats (dd/mm/yyyy, dd-mm-yyyy, etc.)
        return datetime.strptime(re.search(r'\d{1,2}[\/\-]\d{1,2}[\/\-]\d{2,4}', date_str).group(), '%d/%m/%Y')
    except ValueError:
        try:
            return datetime.strptime(re.search(r'\d{1,2}[\/\-]\d{1,2}[\/\-]\d{2,4}', date_str).group(), '%d-%m-%Y')
        except ValueError:
            return None

# Function to calculate expiry date from packing date and 'Best before N months'
def calculate_expiry_from_best_before(packing_date, best_before_months):
    if packing_date and best_before_months:
        # Add N months to the packing date
        return packing_date + relativedelta(months=best_before_months)
    return None

# Function to extract text from front and back images of a product
def extract_product_details(product_folder):
    front_image_path = os.path.join(product_folder, 'front.jpg')
    back_image_path = os.path.join(product_folder, 'back.jpg')

    if not os.path.exists(front_image_path) or not os.path.exists(back_image_path):
        print(f"Images missing in {product_folder}")
        return None

    # Reading text from front and back images
    front_text = reader.readtext(front_image_path, detail=0)
    back_text = reader.readtext(back_image_path, detail=0)

    # Extract MRP, expiry date, best-before details from both front and back
    mrp_front, expiry_front, best_before_front, packing_date_front = extract_mrp_and_expiry(front_text)
    mrp_back, expiry_back, best_before_back, packing_date_back = extract_mrp_and_expiry(back_text)

    # Choose MRP from front or back
    mrp = mrp_front if mrp_front else mrp_back

    # Choose direct expiry date if available
    expiry_date = expiry_front if expiry_front else expiry_back

    # If no direct expiry date, calculate from best-before and packing date
    if not expiry_date:
        packing_date = packing_date_front if packing_date_front else packing_date_back
        best_before_months = best_before_front if best_before_front else best_before_back
        expiry_date = calculate_expiry_from_best_before(packing_date, best_before_months)

    # Combine details
    combined_details = {
        "product_name": os.path.basename(product_folder),
        "front_image_text": front_text,
        "back_image_text": back_text,
        "mrp": mrp,
        "expiry_date": expiry_date.strftime('%d/%m/%Y') if expiry_date else None
    }

    return combined_details

# Iterate through the dataset and process each product folder
def process_dataset(dataset_path):
    product_details = []

    # List all product folders
    for product_name in os.listdir(dataset_path):
        product_folder = os.path.join(dataset_path, product_name)
        if os.path.isdir(product_folder):
            details = extract_product_details(product_folder)
            if details:
                product_details.append(details)

    return product_details

# Run the function on your dataset
product_data = process_dataset(dataset_path)

# Example of how you can display the extracted data
for product in product_data:
    print(f"Product: {product['product_name']}")
    print(f"Front Image Text: {product['front_image_text']}")
    print(f"Back Image Text: {product['back_image_text']}")
    print(f"MRP: {product['mrp']}")
    print(f"Expiry Date: {product['expiry_date']}")
    print("-" * 50)


Images missing in /content/drive/MyDrive/dataset/complain
Product: bread
Front Image Text: ['Huvzaa|', 'ZERO', 'TRANS', 'FAT', '4GLIS,', 'FIBER HELPS', 'DIGESTION', 'OVE)', 'BROWN', 'HIGH', 'FIBRE BREAD', 'FRESHLY BAKED']
Back Image Text: ['Huvzaa|', 'COMMODITY NAME: BROWN BREAD', 'NUTRITONAL INFORMATION', '7.1.1 [Bread and Rolls]', 'PER 100g PRODUCT (APPROX ,', 'INGREDIENTS: Weat Roulma da]', 'Waler   Yeast  Whzat', 'Eneagy', 'Zjoacal', 'Ala(5%0} Sugar', 'Edibe? Conmcn Salt, Ecble Vegetatle', 'carbohydaATES', '53.t1g', 'Ois (Palm}', 'Guren', 'Bran, Roasted Cerea', 'Qf WHICH SUGARS', '2.059', 'Flour;   Soya  Flur;   Class', 'Preservabive/2821,', 'Acidily', 'Reg lator (260]  Emulstiers [471, 472e, 481f], Improve 5', 'SATURATED FAT', '0,26g', '[170 @,510,923,1100] And  Antiowdart (3001.', 'TRANS FaT', 'PROTEIN', '7,930', '(NUMBERS IN BRACKETS ARE AS PER', 'dietary fiber', 'INTERNATIONAL NUMBERING SYSTEM)', 'ALLERGEN DECLARATION: PRODUCT CONTAINS WHEAT FLOUR', '(GLUTEN) AND SOYA FLOUR AS 

In [12]:
!sudo apt install tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 2s (2,957 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debc

In [13]:
!sudo apt install tesseract-ocr-eng

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr-eng is already the newest version (1:4.00~git30-7274cfa-1.1).
tesseract-ocr-eng set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [15]:
import os
import cv2
import pytesseract
import re
from datetime import datetime, timedelta

# Pre-process the images to improve OCR accuracy
def preprocess_image(image_path):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Apply thresholding to clean up the image
    _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)
    return thresh

# Extract text using Tesseract OCR
def extract_text(image_path):
    processed_img = preprocess_image(image_path)
    text = pytesseract.image_to_string(processed_img)
    return text

# Function to find MRP, manufacturing, and expiry date
def find_mrp_and_dates(text):
    mrp_pattern = re.compile(r'Rs\.\s?(\d+\.?\d*)', re.IGNORECASE)
    date_pattern = re.compile(r'(\d{2}/\d{2}/\d{2,4})')
    best_before_pattern = re.compile(r'Best\s+before\s+(\d+)\s+months', re.IGNORECASE)

    # Find all matches
    mrp_match = mrp_pattern.search(text)
    dates = date_pattern.findall(text)
    best_before_match = best_before_pattern.search(text)

    mrp = None
    manufacturing_date = None
    expiry_date = None

    if mrp_match:
        mrp = mrp_match.group(1)

    # Check for expiry and manufacturing dates in normal format
    if len(dates) >= 2:
        manufacturing_date = dates[0]
        expiry_date = dates[1]

    # Handle "best before n months" case
    if best_before_match and manufacturing_date:
        months = int(best_before_match.group(1))
        try:
            manu_date_obj = datetime.strptime(manufacturing_date, '%d/%m/%y')
            expiry_date = (manu_date_obj + timedelta(days=months*30)).strftime('%d/%m/%Y')
        except ValueError:
            print(f"Error processing manufacturing date: {manufacturing_date}")

    return {
        "MRP": mrp,
        "Manufacturing Date": manufacturing_date,
        "Expiry Date": expiry_date
    }

# Process a product directory containing front and back images
def process_product(product_dir):
    front_image_path = os.path.join(product_dir, "front.jpg")
    back_image_path = os.path.join(product_dir, "back.jpg")

    # Check if both images exist
    if not os.path.exists(front_image_path) or not os.path.exists(back_image_path):
        print(f"Missing images for product {product_dir}")
        return None

    # Extract text from both front and back images
    front_text = extract_text(front_image_path)
    back_text = extract_text(back_image_path)

    # Combine text and find details
    combined_text = front_text + " " + back_text
    product_details = find_mrp_and_dates(combined_text)

    return product_details

# Main function to process the dataset
def process_dataset(dataset_dir):
    for product_name in os.listdir(dataset_dir):
        product_dir = os.path.join(dataset_dir, product_name)
        if os.path.isdir(product_dir):
            print(f"Processing product: {product_name}")
            details = process_product(product_dir)
            if details:
                print(f"Details for {product_name}:")
                print(details)
            else:
                print(f"No details extracted for {product_name}")

# Example: Run on the dataset directory
dataset_dir = "/content/drive/MyDrive/dataset"  # Modify this path to your actual dataset
process_dataset(dataset_dir)


Processing product: bread
Details for bread:
{'MRP': None, 'Manufacturing Date': None, 'Expiry Date': None}
Processing product: cheese
Details for cheese:
{'MRP': None, 'Manufacturing Date': None, 'Expiry Date': None}
Processing product: complain
Missing images for product /content/drive/MyDrive/dataset/complain
No details extracted for complain
Processing product: mutton masala
Missing images for product /content/drive/MyDrive/dataset/mutton masala
No details extracted for mutton masala
